## Create Dataframe

In [22]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [23]:
df = pd.read_csv("insider_book.csv",  encoding="utf-8", sep=";")
df.head()

,Filing Date,date 1y before,better date before,date 1y after,date 1y after better,Period,Quarter,firstofmonth,unratechange,unrate,m2supply,m2supplychange,fedfunds,fedfundschange,WGS3MO,WGS3MOchange,WGS10YR,WGS10YRchange,WBAA,WBAAchange,DEXUSEU,DEXUSEUchange,DEXCHUS,DEXCHUSchange,SPCS20RSA,SPCS20RSAchange,INDPRO,INDPROchange,TCU,TCUchange,BUSINV,BUSINVchange,DGORDER,DGORDERchange,BUSLOANS,BUSLOANSchange,TOTALSL,TOTALSLchange,HOUST,HOUSTchange,UNDCONTSA,UNDCONTSAchange,PSAVERT,PSAVERTchange,BOGMBASE,BOGMBASEchange,ticker,Company Name,Industry,Insiders,Own Change,Value Traded,Price,1y before price,1ybeforepricechange,1yafterpricechange,target,pe,1ychangepe,ps,pocf,pfcf,pb,ptb,evs,evebitda,evocf,evfcf,debttoequity,debttoassets,netdebtebitda,currentratio,interestcoverage,capexocf,capexsales,sbcsales,roic,roe,salesgrowth,ebitgrowth,fcfgrowth,3ysalesgrowth,3yocfgrowth,3yequitygrowth,inventorygrowth,assetgrowth,debtgrowth
0,31/03/2022,31/03/2021,31/03/2021,31/03/2023,03/04/2023,2022,Q1,01/03/2022,"-40,98","3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",SPLP,"Steel Partners Holdings L,P,",Miscellaneous Primary Metal Products,4,4%,5459623,"$41,56","13,86","199,90%","$42,01","1,10%","25,13","1611,00%","2,274849945","-69,34703161","-43,83591331","-6,97864081","-6,97864081","3,036249346","19,36743214","-92,55770028","-58,50793079","-14,60528345","0,747007772","4,856765552","1,745139481","10,92970822","0,581968445","-0,019090808","0,000293288","0,013330652","-0,069437934","-0,060464459","-0,212166598","-0,502068769","0,172789964","-1,406007068",0,"0,069017914","-0,033372844","-0,193090854"
1,31/03/2022,31/03/2021,31/03/2021,31/03/2023,03/04/2023,2022,Q1,01/03/2022,"-40,98","3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",JELD,"Jeld-Wen Holding Inc,",Millwood Veneer Plywood & Structural Wood Members,3,29%,60237960,"$20,28","28,37","-28,50%","$11,73","-42,20%","-974,36","-3716,40%","1,664154172","-10,42656252","-9,586405627","2,40049832","2,40049832","3,167377059","209,5479399","-19,84482898","-18,24576218","3,725055432","0,788362271","99,45050847","2,02162696","-0,807880435","0,087640449","-0,013988044","0,008252775","-0,00366373","-0,000615915","-0,090086842","-1,216290541","-15,09708738","0,299303917","-6,493594895","0,191256361","0,095181429","0,025979553","0,057094479"
2,31/03/2022,31/03/2021,31/03/2021,31/03/2023,03/04/2023,2022,Q1,01/03/2022,"-40,98","3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",PMD,Psychemedics Corp,Medical Laboratories,3,2%,192499,"$6,90","6,25","10,40%","$5,48","-20,60%","247,25","139,10%","5,926705593","111,8","127,718543","3,095337453","3,095337453","6,328211432","48,45176471","119,373913","136,3708609","0,682689993","0,405713468","3,074117647","1,97689707","0,151515152","-0,124637681","-0,006607253","0,02750461","-0,000435486","0,003129765","0,011187073","-1,090909091","-0,435514019","-0,347243984","-0,076419946","-0,336182351",0,"-0,027097253","-0,075482738"
3,31/03/2022,31/03/2021,31/03/2021,31/03/2023,03/04/2023,2022,Q1,01/03/2022,"-40,98","3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5

In [24]:
df.columns

Index(['Filing Date', 'date 1y before', 'better date before', 'date 1y after',
       'date 1y after better', 'Period', 'Quarter', 'firstofmonth',
       ' unratechange ', 'unrate', 'm2supply', 'm2supplychange', 'fedfunds',
       'fedfundschange', 'WGS3MO', 'WGS3MOchange', 'WGS10YR', 'WGS10YRchange',
       'WBAA', 'WBAAchange', 'DEXUSEU', 'DEXUSEUchange', 'DEXCHUS',
       'DEXCHUSchange', 'SPCS20RSA', 'SPCS20RSAchange', 'INDPRO',
       'INDPROchange', 'TCU', 'TCUchange', 'BUSINV', 'BUSINVchange', 'DGORDER',
       'DGORDERchange', 'BUSLOANS', 'BUSLOANSchange', 'TOTALSL',
       'TOTALSLchange', 'HOUST', 'HOUSTchange', 'UNDCONTSA', 'UNDCONTSAchange',
       'PSAVERT', 'PSAVERTchange', 'BOGMBASE', 'BOGMBASEchange', 'ticker',
       'Company Name', 'Industry', 'Insiders', 'Own Change', 'Value Traded',
       'Price', '1y before price', '1ybeforepricechange',
       ' 1yafterpricechange ', 'target', ' pe ', '1ychangepe', 'ps', 'pocf',
       'pfcf', 'pb', 'ptb', 'evs', 'evebitda', 'evo

In [25]:
df.shape

(1527, 87)

#### Remove unnecessary features

In [26]:
df.drop(["Industry", "Filing Date", "date 1y before", " unratechange ", "better date before", "firstofmonth", "date 1y after", "date 1y after better", "Period", "Quarter", "ticker", "Company Name", "1y before price", " 1yafterpricechange "], axis=1, inplace=True)
df.head()

,unrate,m2supply,m2supplychange,fedfunds,fedfundschange,WGS3MO,WGS3MOchange,WGS10YR,WGS10YRchange,WBAA,WBAAchange,DEXUSEU,DEXUSEUchange,DEXCHUS,DEXCHUSchange,SPCS20RSA,SPCS20RSAchange,INDPRO,INDPROchange,TCU,TCUchange,BUSINV,BUSINVchange,DGORDER,DGORDERchange,BUSLOANS,BUSLOANSchange,TOTALSL,TOTALSLchange,HOUST,HOUSTchange,UNDCONTSA,UNDCONTSAchange,PSAVERT,PSAVERTchange,BOGMBASE,BOGMBASEchange,Insiders,Own Change,Value Traded,Price,1ybeforepricechange,target,pe,1ychangepe,ps,pocf,pfcf,pb,ptb,evs,evebitda,evocf,evfcf,debttoequity,debttoassets,netdebtebitda,currentratio,interestcoverage,capexocf,capexsales,sbcsales,roic,roe,salesgrowth,ebitgrowth,fcfgrowth,3ysalesgrowth,3yocfgrowth,3yequitygrowth,inventorygrowth,assetgrowth,debtgrowth
0,"3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",4,4%,5459623,"$41,56","199,90%","1,10%","25,13","1611,00%","2,274849945","-69,34703161","-43,83591331","-6,97864081","-6,97864081","3,036249346","19,36743214","-92,55770028","-58,50793079","-14,60528345","0,747007772","4,856765552","1,745139481","10,92970822","0,581968445","-0,019090808","0,000293288","0,013330652","-0,069437934","-0,060464459","-0,212166598","-0,502068769","0,172789964","-1,406007068",0,"0,069017914","-0,033372844","-0,193090854"
1,"3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",3,29%,60237960,"$20,28","-28,50%","-42,20%","-974,36","-3716,40%","1,664154172","-10,42656252","-9,586405627","2,40049832","2,40049832","3,167377059","209,5479399","-19,84482898","-18,24576218","3,725055432","0,788362271","99,45050847","2,02162696","-0,807880435","0,087640449","-0,013988044","0,008252775","-0,00366373","-0,000615915","-0,090086842","-1,216290541","-15,09708738","0,299303917","-6,493594895","0,191256361","0,095181429","0,025979553","0,057094479"
2,"3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",3,2%,192499,"$6,90","10,40%","-20,60%","247,25","139,10%","5,926705593","111,8","127,718543","3,095337453","3,095337453","6,328211432","48,45176471","119,373913","136,3708609","0,682689993","0,405713468","3,074117647","1,97689707","0,151515152","-0,124637681","-0,006607253","0,02750461","-0,000435486","0,003129765","0,011187073","-1,090909091","-0,435514019","-0,347243984","-0,076419946","-0,336182351",0,"-0,027097253","-0,075482738"
3,"3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,43","80,52","5,60","2.314.760,00","15,05","264.258,00","9,47","2.531,28","-2,16","4.522,07","6,99","1.716,00","0,29","1.629,00","24,73","3,80","-85,55","6.134.400,00","5,06",5,32%,1043463,"$13,13","-49,60%","-27,90%","-4,45","-116,20%","2,320730378","20,00004341","29,31180451","0,512443099","0,512443099","2,971866013","-112,0891364","25,61152723","37,5359225","0,368411207","0,269225512","-24,55872193","6,089422067","-29,89784946","-0,317679558","-0,03686235","0,016347651","-0,025881891","-0,028792","-0,048308021","-0,901846785","-0,455947137","0,358044284","5,160776343",0,"-0,008587888","-0,025809819","-0,073987413"
4,"3,60","21.697,70","9,36","0,20","185,71","0,42","1.300,00","2,06","29,36","4,29","14,65","1,10","-7,42","6,34","-2,55","305,90","21,18","102,48","4,4

#### Transform values to correct types

In [27]:
df.dtypes

unrate             object
m2supply           object
m2supplychange     object
fedfunds           object
fedfundschange     object
                    ...  
3yocfgrowth        object
3yequitygrowth     object
inventorygrowth    object
assetgrowth        object
debtgrowth         object
Length: 73, dtype: object

In [28]:
df = df.astype(str)

In [29]:
# change % values to floats
df=  df.apply(lambda x: x.str.replace(",",".").str.replace("%","").str.replace("$",""))

df.head()

C:\Users\Nils\AppData\Local\Temp\ipykernel_24396\3887205652.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,unrate,m2supply,m2supplychange,fedfunds,fedfundschange,WGS3MO,WGS3MOchange,WGS10YR,WGS10YRchange,WBAA,WBAAchange,DEXUSEU,DEXUSEUchange,DEXCHUS,DEXCHUSchange,SPCS20RSA,SPCS20RSAchange,INDPRO,INDPROchange,TCU,TCUchange,BUSINV,BUSINVchange,DGORDER,DGORDERchange,BUSLOANS,BUSLOANSchange,TOTALSL,TOTALSLchange,HOUST,HOUSTchange,UNDCONTSA,UNDCONTSAchange,PSAVERT,PSAVERTchange,BOGMBASE,BOGMBASEchange,Insiders,Own Change,Value Traded,Price,1ybeforepricechange,target,pe,1ychangepe,ps,pocf,pfcf,pb,ptb,evs,evebitda,evocf,evfcf,debttoequity,debttoassets,netdebtebitda,currentratio,interestcoverage,capexocf,capexsales,sbcsales,roic,roe,salesgrowth,ebitgrowth,fcfgrowth,3ysalesgrowth,3yocfgrowth,3yequitygrowth,inventorygrowth,assetgrowth,debtgrowth
0,3.60,21.697.70,9.36,0.20,185.71,0.42,1.300.00,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314.760.00,15.05,264.258.00,9.47,2.531.28,-2.16,4.522.07,6.99,1.716.00,0.29,1.629.00,24.73,3.80,-85.55,6.134.400.00,5.06,4,4,5459623,41.56,199.90,1.10,25.13,1611.00,2.274849945,-69.34703161,-43.83591331,-6.97864081,-6.97864081,3.036249346,19.36743214,-92.55770028,-58.50793079,-14.60528345,0.747007772,4.856765552,1.745139481,10.92970822,0.581968445,-0.019090808,0.000293288,0.013330652,-0.069437934,-0.060464459,-0.212166598,-0.502068769,0.172789964,-1.406007068,0,0.069017914,-0.033372844,-0.193090854
1,3.60,21.697.70,9.36,0.20,185.71,0.42,1.300.00,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314.760.00,15.05,264.258.00,9.47,2.531.28,-2.16,4.522.07,6.99,1.716.00,0.29,1.629.00,24.73,3.80,-85.55,6.134.400.00,5.06,3,29,60237960,20.28,-28.50,-42.20,-974.36,-3716.40,1.664154172,-10.42656252,-9.586405627,2.40049832,2.40049832,3.167377059,209.5479399,-19.84482898,-18.24576218,3.725055432,0.788362271,99.45050847,2.02162696,-0.807880435,0.087640449,-0.013988044,0.008252775,-0.00366373,-0.000615915,-0.090086842,-1.216290541,-15.09708738,0.299303917,-6.493594895,0.191256361,0.095181429,0.025979553,0.057094479
2,3.60,21.697.70,9.36,0.20,185.71,0.42,1.300.00,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314.760.00,15.05,264.258.00,9.47,2.531.28,-2.16,4.522.07,6.99,1.716.00,0.29,1.629.00,24.73,3.80,-85.55,6.134.400.00,5.06,3,2,192499,6.90,10.40,-20.60,247.25,139.10,5.926705593,111.8,127.718543,3.095337453,3.095337453,6.328211432,48.45176471,119.373913,136.3708609,0.682689993,0.405713468,3.074117647,1.97689707,0.151515152,-0.124637681,-0.006607253,0.02750461,-0.000435486,0.003129765,0.011187073,-1.090909091,-0.435514019,-0.347243984,-0.076419946,-0.336182351,0,-0.027097253,-0.075482738
3,3.60,21.697.70,9.36,0.20,185.71,0.42,1.300.00,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314.760.00,15.05,264.258.00,9.47,2.531.28,-2.16,4.522.07,6.99,1.716.00,0.29,1.629.00,24.73,3.80,-85.55,6.134.400.00,5.06,5,32,1043463,13.13,-49.60,-27.90,-4.45,-116.20,2.320730378,20.00004341,29.31180451,0.512443099,0.512443099,2.971866013,-112.0891364,25.61152723,37.5359225,0.368411207,0.269225512,-24.55872193,6.089422067,-29.89784946,-0.317679558,-0.03686235,0.016347651,-0.025881891,-0.028792,-0.048308021,-0.901846785,-0.455947137,0.358044284,5.160776343,0,-0.008587888,-0.025809819,-0.073987413
4,3.60,21.697.70,9.36,0.20,185.71,0.42,1.300.00,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314.760.00,15.05,264.258.00,9.47,2.531.28,-2.16,4.522.07,6.99,1.716.00,0.29,1.629.00,24.73,3.80,-85.55,6.134.400.00,5.06,6,66,157802366,5.85,-81.40,-57.80,-7.19,-129.30,2.739852213,-8.859088548,-6.839193501,1.739603556,1.739603556,3.24844635,-32.0654464,-10.50358619,-8.108741436,1.838823973,0.64695319,-5.020337815,1.887991021,-17.95112414,0.295341117,-0.091340211,0.003785471,-0.035810526,-0.060446553,-0.232000043,0.338401124,-0.230985492,-0.784641898,0.327951924,40.46069572,0.072307225,-0.002367426,-0.265970469


In [30]:
import numpy as np

df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.apply(lambda x: x.str.replace(r'(\d+)\.(\d+)\.', r'\1.\2').str.replace(',', ''))

df.head(100)

C:\Users\Nils\AppData\Local\Temp\ipykernel_24396\2554014945.py:4: FutureWarning: The default value of regex will change from True to False in a future version.


,unrate,m2supply,m2supplychange,fedfunds,fedfundschange,WGS3MO,WGS3MOchange,WGS10YR,WGS10YRchange,WBAA,WBAAchange,DEXUSEU,DEXUSEUchange,DEXCHUS,DEXCHUSchange,SPCS20RSA,SPCS20RSAchange,INDPRO,INDPROchange,TCU,TCUchange,BUSINV,BUSINVchange,DGORDER,DGORDERchange,BUSLOANS,BUSLOANSchange,TOTALSL,TOTALSLchange,HOUST,HOUSTchange,UNDCONTSA,UNDCONTSAchange,PSAVERT,PSAVERTchange,BOGMBASE,BOGMBASEchange,Insiders,Own Change,Value Traded,Price,1ybeforepricechange,target,pe,1ychangepe,ps,pocf,pfcf,pb,ptb,evs,evebitda,evocf,evfcf,debttoequity,debttoassets,netdebtebitda,currentratio,interestcoverage,capexocf,capexsales,sbcsales,roic,roe,salesgrowth,ebitgrowth,fcfgrowth,3ysalesgrowth,3yocfgrowth,3yequitygrowth,inventorygrowth,assetgrowth,debtgrowth
0,3.60,21.69770,9.36,0.20,185.71,0.42,1.30000,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314760.00,15.05,264.25800,9.47,2.53128,-2.16,4.52207,6.99,1.71600,0.29,1.62900,24.73,3.80,-85.55,6.134400.00,5.06,4,4,5459623,41.56,199.90,1.10,25.13,1611.00,2.274849945,-69.34703161,-43.83591331,-6.97864081,-6.97864081,3.036249346,19.36743214,-92.55770028,-58.50793079,-14.60528345,0.747007772,4.856765552,1.745139481,10.92970822,0.581968445,-0.019090808,0.000293288,0.013330652,-0.069437934,-0.060464459,-0.212166598,-0.502068769,0.172789964,-1.406007068,0,0.069017914,-0.033372844,-0.193090854
1,3.60,21.69770,9.36,0.20,185.71,0.42,1.30000,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314760.00,15.05,264.25800,9.47,2.53128,-2.16,4.52207,6.99,1.71600,0.29,1.62900,24.73,3.80,-85.55,6.134400.00,5.06,3,29,60237960,20.28,-28.50,-42.20,-974.36,-3716.40,1.664154172,-10.42656252,-9.586405627,2.40049832,2.40049832,3.167377059,209.5479399,-19.84482898,-18.24576218,3.725055432,0.788362271,99.45050847,2.02162696,-0.807880435,0.087640449,-0.013988044,0.008252775,-0.00366373,-0.000615915,-0.090086842,-1.216290541,-15.09708738,0.299303917,-6.493594895,0.191256361,0.095181429,0.025979553,0.057094479
2,3.60,21.69770,9.36,0.20,185.71,0.42,1.30000,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314760.00,15.05,264.25800,9.47,2.53128,-2.16,4.52207,6.99,1.71600,0.29,1.62900,24.73,3.80,-85.55,6.134400.00,5.06,3,2,192499,6.90,10.40,-20.60,247.25,139.10,5.926705593,111.8,127.718543,3.095337453,3.095337453,6.328211432,48.45176471,119.373913,136.3708609,0.682689993,0.405713468,3.074117647,1.97689707,0.151515152,-0.124637681,-0.006607253,0.02750461,-0.000435486,0.003129765,0.011187073,-1.090909091,-0.435514019,-0.347243984,-0.076419946,-0.336182351,0,-0.027097253,-0.075482738
3,3.60,21.69770,9.36,0.20,185.71,0.42,1.30000,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314760.00,15.05,264.25800,9.47,2.53128,-2.16,4.52207,6.99,1.71600,0.29,1.62900,24.73,3.80,-85.55,6.134400.00,5.06,5,32,1043463,13.13,-49.60,-27.90,-4.45,-116.20,2.320730378,20.00004341,29.31180451,0.512443099,0.512443099,2.971866013,-112.0891364,25.61152723,37.5359225,0.368411207,0.269225512,-24.55872193,6.089422067,-29.89784946,-0.317679558,-0.03686235,0.016347651,-0.025881891,-0.028792,-0.048308021,-0.901846785,-0.455947137,0.358044284,5.160776343,0,-0.008587888,-0.025809819,-0.073987413
4,3.60,21.69770,9.36,0.20,185.71,0.42,1.30000,2.06,29.36,4.29,14.65,1.10,-7.42,6.34,-2.55,305.90,21.18,102.48,4.43,80.52,5.60,2.314760.00,15.05,264.25800,9.47,2.53128,-2.16,4.52207,6.99,1.71600,0.29,1.62900,24.73,3.80,-85.55,6.134400.00,5.06,6,66,157802366,5.85,-81.40,-57.80,-7.19,-129.30,2.739852213,-8.859088548,-6.839193501,1.739603556,1.739603556,3.24844635,-32.0654464,-10.50358619,-8.108741436,1.838823973,0.64695319,-5.020337815,1.887991021,-17.95112414,0.295341117,-0.091340211,0.003785471,-0.035810526,-0.060446553,-0.232000043,0.338401124,-0.230985492,-0.784641898,0.327951924,40.46069572,0.072307225,-0.002367426,-0.265970469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [31]:
# convert columns to float
for col in df.columns:
    try:
        df[col] = df[col].astype(float)
    except ValueError:
        # replace non-numeric values with NaN
        df[col] = df[col].replace('-', pd.NaT)

In [32]:
print(df.dtypes)

unrate             float64
m2supply           float64
m2supplychange     float64
fedfunds           float64
fedfundschange      object
                    ...   
3yocfgrowth        float64
3yequitygrowth     float64
inventorygrowth    float64
assetgrowth        float64
debtgrowth         float64
Length: 73, dtype: object


In [33]:
# change target feature to binary (1 if > 20 , 0 if <= 2)
df["target"] = df["target"].apply(lambda x: 1 if x < -50 else 0)

df.head()

,unrate,m2supply,m2supplychange,fedfunds,fedfundschange,WGS3MO,WGS3MOchange,WGS10YR,WGS10YRchange,WBAA,WBAAchange,DEXUSEU,DEXUSEUchange,DEXCHUS,DEXCHUSchange,SPCS20RSA,SPCS20RSAchange,INDPRO,INDPROchange,TCU,TCUchange,BUSINV,BUSINVchange,DGORDER,DGORDERchange,BUSLOANS,BUSLOANSchange,TOTALSL,TOTALSLchange,HOUST,HOUSTchange,UNDCONTSA,UNDCONTSAchange,PSAVERT,PSAVERTchange,BOGMBASE,BOGMBASEchange,Insiders,Own Change,Value Traded,Price,1ybeforepricechange,target,pe,1ychangepe,ps,pocf,pfcf,pb,ptb,evs,evebitda,evocf,evfcf,debttoequity,debttoassets,netdebtebitda,currentratio,interestcoverage,capexocf,capexsales,sbcsales,roic,roe,salesgrowth,ebitgrowth,fcfgrowth,3ysalesgrowth,3yocfgrowth,3yequitygrowth,inventorygrowth,assetgrowth,debtgrowth
0,3.6,21.6977,9.36,0.2,185.71,0.42,1.3,2.06,29.36,4.29,14.65,1.1,-7.42,6.34,-2.55,305.9,21.18,102.48,4.43,80.52,5.6,2.314760.00,15.05,264.258,9.47,2.53128,-2.16,4.52207,6.99,1.716,0.29,1.629,24.73,3.8,-85.55,6.134400.00,5.06,4.0,4,5459623.0,41.56,199.9,0,25.13,1611.0,2.274850,-69.347032,-43.835913,-6.978641,-6.978641,3.036249,19.367432,-92.557700,-58.507931,-14.605283,0.747008,4.856766,1.745139,10.929708,0.581968,-0.019091,0.000293,0.013331,-0.069438,-0.060464,-0.212167,-0.502069,0.172790,-1.406007,0.000000,0.069018,-0.033373,-0.193091
1,3.6,21.6977,9.36,0.2,185.71,0.42,1.3,2.06,29.36,4.29,14.65,1.1,-7.42,6.34,-2.55,305.9,21.18,102.48,4.43,80.52,5.6,2.314760.00,15.05,264.258,9.47,2.53128,-2.16,4.52207,6.99,1.716,0.29,1.629,24.73,3.8,-85.55,6.134400.00,5.06,3.0,29,60237960.0,20.28,-28.5,0,-974.36,-3716.4,1.664154,-10.426563,-9.586406,2.400498,2.400498,3.167377,209.547940,-19.844829,-18.245762,3.725055,0.788362,99.450508,2.021627,-0.807880,0.087640,-0.013988,0.008253,-0.003664,-0.000616,-0.090087,-1.216291,-15.097087,0.299304,-6.493595,0.191256,0.095181,0.025980,0.057094
2,3.6,21.6977,9.36,0.2,185.71,0.42,1.3,2.06,29.36,4.29,14.65,1.1,-7.42,6.34,-2.55,305.9,21.18,102.48,4.43,80.52,5.6,2.314760.00,15.05,264.258,9.47,2.53128,-2.16,4.52207,6.99,1.716,0.29,1.629,24.73,3.8,-85.55,6.134400.00,5.06,3.0,2,192499.0,6.90,10.4,0,247.25,139.1,5.926706,111.800000,127.718543,3.095337,3.095337,6.328211,48.451765,119.373913,136.370861,0.682690,0.405713,3.074118,1.976897,0.151515,-0.124638,-0.006607,0.027505,-0.000435,0.003130,0.011187,-1.090909,-0.435514,-0.347244,-0.076420,-0.336182,0.000000,-0.027097,-0.075483
3,3.6,21.6977,9.36,0.2,185.71,0.42,1.3,2.06,29.36,4.29,14.65,1.1,-7.42,6.34,-2.55,305.9,21.18,102.48,4.43,80.52,5.6,2.314760.00,15.05,264.258,9.47,2.53128,-2.16,4.52207,6.99,1.716,0.29,1.629,24.73,3.8,-85.55,6.134400.00,5.06,5.0,32,1043463.0,13.13,-49.6,0,-4.45,-116.2,2.320730,20.000043,29.311805,0.512443,0.512443,2.971866,-112.089136,25.611527,37.535922,0.368411,0.269226,-24.558722,6.089422,-29.897849,-0.317680,-0.036862,0.016348,-0.025882,-0.028792,-0.048308,-0.901847,-0.455947,0.358044,5.160776,0.000000,-0.008588,-0.025810,-0.073987
4,3.6,21.6977,9.36,0.2,185.71,0.42,1.3,2.06,29.36,4.29,14.65,1.1,-7.42,6.34,-2.55,305.9,21.18,102.48,4.43,80.52,5.6,2.314760.00,15.05,264.258,9.47,2.53128,-2.16,4.52207,6.99,1.716,0.29,1.629,24.73,3.8,-85.55,6.134400.00,5.06,6.0,66,157802366.0,5.85,-81.4,1,-7.19,-129.3,2.739852,-8.859089,-6.839194,1.739604,1.739604,3.248446,-32.065446,-10.503586,-8.108741,1.838824,0.646953,-5.020338,1.887991,-17.951124,0.295341,-0.091340,0.003785,-0.035811,-0.060447,-0.232000,0.338401,-0.230985,-0.784642,0.327952,40.460696,0.072307,-0.002367,-0.265970


In [34]:
df.shape

(1527, 73)

#### Run AutoML

In [35]:
from supervised.automl import AutoML
from sklearn.metrics import log_loss

X = df.drop("target", axis=1)
y = df["target"]

automl = AutoML(eval_metric="average_precision", results_path = "final_model_short", features_selection=True, golden_features=True, kmeans_features=True, train_ensemble=True)
automl.fit(X, y)

AutoML directory: final_model_short
The task is binary_classification with evaluation metric average_precision
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline average_precision 0.143979 trained in 0.24 seconds
2_DecisionTree average_precision 0.413238 trained in 9.72 seconds
3_Linear average_precision 0.364799 trained in 2.5 seconds
* Step default_algorithms will try to check up to 3 models


C:\Users\Nils\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.


4_Default_Xgboost average_precision 0.577354 trained in 3.39 seconds
5_Default_NeuralNetwork average_precision 0.419834 trained in 0.95 seconds
6_Default_RandomForest average_precision 0.600561 trained in 6.96 seconds
* Step golden_features will try to check up to 3 models
None 10
Add Golden Feature: WGS10YRchange_diff_assetgrowth
Add Golden Feature: BOGMBASEchange_ratio_WGS10YRchange
Add Golden Feature: WGS10YRchange_ratio_BOGMBASEchange
Add Golden Feature: WGS3MOchange_diff_WGS10YRchange
Add Golden Feature: TCUchange_ratio_WBAA
Add Golden Feature:  pe _multiply_TCUchange
Add Golden Feature: UNDCONTSA_multiply_WGS10YR
Add Golden Feature: TCUchange_ratio_TOTALSLchange
Add Golden Feature: WGS10YR_ratio_WBAA
Add Golden Feature: WBAA_ratio_WGS10YR
Created 10 Golden Features in 1.96 seconds.
6_Default_RandomForest_GoldenFeatures average_precision 0.570392 trained in 6.67 seconds


C:\Users\Nils\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.


4_Default_Xgboost_GoldenFeatures average_precision 0.617838 trained in 3.32 seconds
5_Default_NeuralNetwork_GoldenFeatures average_precision 0.476102 trained in 1.3 seconds
* Step kmeans_features will try to check up to 3 models


...

C:\Users\Nils\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.


4_Default_Xgboost_GoldenFeatures_RandomFeature average_precision 0.588996 trained in 2.27 seconds
Drop features ['HOUST', 'BUSINVchange', 'ptb', 'UNDCONTSA', 'WBAA_ratio_WGS10YR', 'TOTALSL', 'DGORDER', 'UNDCONTSA_multiply_WGS10YR', 'TOTALSLchange', 'BUSINV', 'TCU', 'INDPROchange', 'fedfunds', 'TCUchange', 'SPCS20RSA', 'TCUchange_ratio_WBAA', 'DEXUSEU', 'BOGMBASE', 'WGS3MO', 'HOUSTchange', 'SPCS20RSAchange', 'm2supply', 'WGS10YR', 'BUSLOANS', 'PSAVERT', 'netdebtebitda', 'debtgrowth', 'UNDCONTSAchange', 'WBAAchange', 'pfcf', 'debttoequity', 'sbcsales', 'Insiders', 'random_feature', 'Own Change', 'currentratio']
* Step features_selection will try to check up to 3 models


C:\Users\Nils\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:39: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.


4_Default_Xgboost_GoldenFeatures_SelectedFeatures average_precision 0.674537 trained in 3.82 seconds
6_Default_RandomForest_SelectedFeatures average_precision 0.593792 trained in 4.02 seconds
5_Default_NeuralNetwork_GoldenFeatures_SelectedFeatures average_precision 0.322288 trained in 1.48 seconds
* Step ensemble will try to check up to 1 model
Ensemble average_precision 0.674537 trained in 0.47 seconds
AutoML fit time: 70.89 seconds
AutoML best model: 4_Default_Xgboost_GoldenFeatures_SelectedFeatures


AutoML(eval_metric='average_precision', features_selection=True,
       golden_features=True, kmeans_features=True,
       results_path='final_model_short')

In [36]:
automl._best_model

In [37]:
import joblib

joblib.dump(automl, "final_short_model.pkl")

['final_short_model.pkl']

In [38]:
automl.get_params()

{'algorithms': 'auto',
 'boost_on_errors': 'auto',
 'eval_metric': 'average_precision',
 'explain_level': 'auto',
 'features_selection': True,
 'golden_features': True,
 'hill_climbing_steps': 'auto',
 'kmeans_features': True,
 'max_single_prediction_time': None,
 'mix_encoding': 'auto',
 'ml_task': 'auto',
 'mode': 'Explain',
 'model_time_limit': None,
 'n_jobs': -1,
 'optuna_init_params': {},
 'optuna_time_budget': None,
 'optuna_verbose': True,
 'random_state': 1234,
 'results_path': 'final_model_short',
 'stack_models': 'auto',
 'start_random_models': 'auto',
 'top_models_to_improve': 'auto',
 'total_time_limit': 3600,
 'train_ensemble': True,
 'validation_strategy': 'auto',
 'verbose': 1}

In [39]:
automl.get_leaderboard()

,name,model_type,metric_type,metric_value,train_time
0,1_Baseline,Baseline,average_precision,-0.143979,0.68
1,2_DecisionTree,Decision Tree,average_precision,-0.413238,10.30
2,3_Linear,Linear,average_precision,-0.364799,3.08
3,4_Default_Xgboost,Xgboost,average_precision,-0.577354,3.98
4,5_Default_NeuralNetwork,Neural Network,average_precision,-0.419834,1.44
5,6_Default_RandomForest,Random Forest,average_precision,-0.600561,7.56
6,6_Default_RandomForest_GoldenFeatures,Random Forest,average_precision,-0.570392,7.27
7,4_Default_Xgboost_GoldenFeatures,Xgboost,average_precision,-0.617838,3.92
8,5_Default_NeuralNetwork_GoldenFeatures,Neural Network,average_precision,-0.476102,1.88
9,6_Default_RandomForest_KMeansFeatures,Random Forest,average_precision,-0.566507,6.04


In [40]:
automl._get_eval_metric()

'average_precision'

In [41]:
automl._get_features_selection()

True

In [42]:
automl._get_golden_features()

True